In [1]:
# https://www.youtube.com/watch?v=3QiPPX-KeSc

import socket
import threading
import random
import string
import time

HEADER = 64
PORT = 8080
SERVER = "0.0.0.0"
ADDR = (SERVER, PORT)
FORMAT = 'utf-8'
DISCONNECT_MESSAGE = "!DISCONNECT"

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(ADDR)

connections = []
rooms = []
stack = [[0,0], [1,0], [1,0], [2,0], [2,0], [3,0], [3,0], [4,0], [4,0], [5,0], [5,0], [6,0], [6,0], [7,0], [7,0], [8,0], [8,0], [9,0], [9,0], [10,0], [10,0], [11,0], [11,0], [12,0], [12,0], 
             [0,1], [1,1], [1,1], [2,1], [2,1], [3,1], [3,1], [4,1], [4,1], [5,1], [5,1], [6,1], [6,1], [7,1], [7,1], [8,1], [8,1], [9,1], [9,1], [10,1], [10,1], [11,1], [11,1], [12,1], [12,1], 
             [0,2], [1,2], [1,2], [2,2], [2,2], [3,2], [3,2], [4,2], [4,2], [5,2], [5,2], [6,2], [6,2], [7,2], [7,2], [8,2], [8,2], [9,2], [9,2], [10,2], [10,2], [11,2], [11,2], [12,2], [12,2], 
             [0,3], [1,3], [1,3], [2,3], [2,3], [3,3], [3,3], [4,3], [4,3], [5,3], [5,3], [6,3], [6,3], [7,3], [7,3], [8,3], [8,3], [9,3], [9,3], [10,3], [10,3], [11,3], [11,3], [12,3], [12,3], 
             [0,4], [0,4], [0,4], [0,4], [5,4], [5,4], [5,4], [5,4]]
stackByRoom = []
heapByRoom = []

In [2]:
def handle_client(conn, addr):
    connections.append(conn)
    print(f"[INFO] New connection, {addr} connected.")
    connected = True
    while connected:
        msg_lenght = conn.recv(HEADER).decode(FORMAT)
        if msg_lenght:
            msg_lenght = int(msg_lenght)
            msg = conn.recv(msg_lenght).decode(FORMAT)
            if msg == DISCONNECT_MESSAGE:
                connected = False
                print(f"[INFO] A client disconnected")
                print(f"[ACTIVE CONNECTION] {threading.activeCount() - 9 }")
                currentRoom = ""
                currentUsername = ""
                for r in rooms:
                    if r[0] == conn:
                        currentRoom = r[1]
                        currentUsername = r[2]
                for r in rooms:
                    if r[1] == currentRoom:
                        if r[0] == conn:
                            print(f"[{addr}] {currentUsername} has been diconnected from room {currentRoom}.")
                            rooms.remove(r)
                counter = 0
                for r in rooms:
                    if r[1] == currentRoom:
                        counter = counter + 1
                if counter > 0:
                    print(f"[INFO] Sending to players in room {currentRoom} updated player list.")
                    messageToSend = f"setPlayers"
                    for r in rooms:
                        if r[1] == currentRoom:
                            messageToSend = messageToSend + "," + str(r[2])
                    for r in rooms:
                        if r[1] == currentRoom and r[0] != conn:
                            r[0].send(messageToSend.encode(FORMAT))
            else:
                myMessage = msg.split(",")
                if myMessage[1] == "createRoom":
                    randomRoomNum = random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits)
                    messageToSend = f"joinNow,{randomRoomNum}"
                    rooms.append([conn, randomRoomNum, myMessage[0]])
                    print(f"[{addr}] {myMessage[0]} created room number {randomRoomNum}")
                    conn.send(messageToSend.encode(FORMAT))
                if myMessage[1] == "joinRoom":
                    counter = 0
                    for r in rooms:
                        if r[1] == myMessage[2]:
                            counter = counter + 1
                            if r[0] == conn:
                                print(f"[{addr}] {myMessage[0]} was already in room number {myMessage[2]}. He has been diconnected.")
                                rooms.remove(r)
                    if counter != 0 and counter < 4:
                        rooms.append([conn, myMessage[2], myMessage[0]])
                        print(f"[{addr}] {myMessage[0]} joined room number {myMessage[2]}. Sending update...")
                        messageToSend = f"joinNow,{myMessage[2]}"
                        conn.send(messageToSend.encode(FORMAT))
                        messageToSend = f"setPlayers"
                        for r in rooms:
                            if r[1] == myMessage[2]:
                                messageToSend = messageToSend + "," + str(r[2])
                        for r in rooms:
                            if r[1] == myMessage[2] and r[0] != conn:
                                r[0].send(messageToSend.encode(FORMAT))
                    else:
                        messageToSend = f"error,roomCodeNotValid"
                        conn.send(messageToSend.encode(FORMAT))
                        print(f"[{addr}] {myMessage[0]} tried to join room number {myMessage[2]}, but room doesn't exist or is full.")
                if myMessage[1] == "getPlayers":
                    print(f"[INFO] {myMessage[0]} want players in room {myMessage[2]}.")
                    messageToSend = f"setPlayers"
                    for r in rooms:
                        if r[1] == myMessage[2]:
                            messageToSend = messageToSend + "," + str(r[2])
                    conn.send(messageToSend.encode(FORMAT))
                if myMessage[1] == "quitRoom":
                    for r in rooms:
                        if r[1] == myMessage[2]:
                            if r[0] == conn:
                                print(f"[{addr}] {myMessage[0]} has been diconnected from room {myMessage[2]}.")
                                rooms.remove(r)
                    counter = 0
                    for r in rooms:
                        if r[1] == myMessage[2]:
                            counter = counter + 1
                    if counter > 0:
                        print(f"[INFO] Sending to players in room {myMessage[2]} updated player list.")
                        messageToSend = f"setPlayers"
                        for r in rooms:
                            if r[1] == myMessage[2]:
                                messageToSend = messageToSend + "," + str(r[2])
                        for r in rooms:
                            if r[1] == myMessage[2] and r[0] != conn:
                                r[0].send(messageToSend.encode(FORMAT))
               	if myMessage[1] == "startGame":
                    counter = 0
                    for r in rooms:
                        if r[1] == myMessage[2]:
                            counter = counter + 1
                    if counter < 0:
                        messageToSend = f"error,notAValidNumberOfPerson"
                        print(f"[INFO] {myMessage[0]} wanted to start room {myMessage[2]}, but there wasn't enough players.")
                        conn.send(messageToSend.encode(FORMAT))
                    else:
                        tempStack = stack
                        random.shuffle(tempStack)
                        stackByRoom = [myMessage[2],tempStack]
                        counter = 0
                        hands = []
                        for r in rooms:
                            if r[1] == myMessage[2]:
                                counter = counter + 1
                        for c in range(0, counter):
                            tmpList = []
                            for i in range(0, 7):
                                tmpList.append(stackByRoom[1].pop(0))
                            hands.append(tmpList)
                        while ((stackByRoom[1][0][1] == 4) | (stackByRoom[1][0][0] in range(10, 13))) : 
                            random.shuffle(stackByRoom[1])
                        heapByRoom = stackByRoom[1].pop(0)
                        print(f"[INFO] {myMessage[0]} started room {myMessage[2]}.")
                        counter = 0
                        for r in rooms:
                            if r[1] == myMessage[2]:
                                messageToSend = f"startNow"
                                messageToSend = messageToSend + "," + str(heapByRoom) + "," + str(hands[counter])
                                counter = counter + 1
                                r[0].send(messageToSend.encode(FORMAT))
                #conn.send(msg.encode(FORMAT))
                # Pour envoyer a tous les clients
                #for connClient in connections:
                    #connClient.send(msg.encode(FORMAT))
    conn.close()
    connections.remove(conn)

In [3]:
def start():
    server.listen()
    print(f"[LISTENING] Server is listening on {SERVER}")
    while True:
        conn, addr = server.accept()
        thread = threading.Thread(target=handle_client, args=(conn, addr))
        thread.start()
        print(f"[ACTIVE CONNECTION] {threading.activeCount() - 8 }")
print("[STARTING] le serveur démarre")

[STARTING] le serveur démarre


In [ ]:
start()

[LISTENING] Server is listening on 0.0.0.0
[INFO] New connection, ('127.0.0.1', 51484) connected.
[ACTIVE CONNECTION] -1
[('127.0.0.1', 51484)] awdsd created room number I5RWT


Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\Eddy\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\Eddy\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Eddy\AppData\Local\Temp\ipykernel_17808\4261884243.py", line 6, in handle_client
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant


[INFO] New connection, ('127.0.0.1', 51512) connected.
[ACTIVE CONNECTION] -1
[('127.0.0.1', 51512)] awdasdw created room number K86HX


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\Eddy\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\Eddy\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Eddy\AppData\Local\Temp\ipykernel_17808\4261884243.py", line 6, in handle_client
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant


[INFO] New connection, ('127.0.0.1', 51542) connected.[ACTIVE CONNECTION] -1

[('127.0.0.1', 51542)] <sccysed created room number H4RB1


Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\Eddy\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\Eddy\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Eddy\AppData\Local\Temp\ipykernel_17808\4261884243.py", line 6, in handle_client
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant


[INFO] New connection, ('127.0.0.1', 51566) connected.[ACTIVE CONNECTION] -1

[('127.0.0.1', 51566)] awdawd created room number O22TP
[INFO] awdawd want players in room O22TP.
[('127.0.0.1', 51566)] awdawd has been diconnected from room O22TP.
[('127.0.0.1', 51566)] awdawd created room number D5P5L


Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\Eddy\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\Eddy\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Eddy\AppData\Local\Temp\ipykernel_17808\4261884243.py", line 6, in handle_client
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant


[INFO] New connection, ('127.0.0.1', 51589) connected.
[ACTIVE CONNECTION] -1
[('127.0.0.1', 51589)] awdawd created room number SX5NY
[INFO] awdawd want players in room SX5NY.
